In [3]:
import os
import sys
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from tenacity import retry, wait_exponential, stop_after_attempt, Retrying, retry_if_exception_type
import google.api_core.exceptions

# Load environment variables
load_dotenv()
PINE_API_KEY = os.environ.get("PINECONE_API_KEY")
GEMINI_API_KEY = os.environ.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

# Pinecone vector store setup
index_name = "therapybot"
# You might need to handle the case where the index doesn't exist
# For example, using a try-except block
try:
    docsearch = PineconeVectorStore.from_existing_index(
        index_name=index_name,
        embedding=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    )
except Exception as e:
    print(f"Pinecone index '{index_name}' does not exist or an error occurred: {e}")
    print("Creating and populating the index...")
    # This part would require your data loading and splitting logic to be here
    loader = DirectoryLoader("C:\\Users\\Shounak Dutta\\Documents\\Sih'25 hearMeout\\hearMeout\\Data",
                             glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    docsearch = PineconeVectorStore.from_documents(
        documents=text_chunks,
        index_name=index_name,
        embedding=embeddings,
    )

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Gemini LLM setup with retry logic
# We use a retry decorator to automatically handle API errors,
# specifically the 429 RateLimitError and 500 Internal Server Errors.
@retry(
    wait=wait_exponential(multiplier=1, min=4, max=10),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(google.api_core.exceptions.ResourceExhausted)
)
def create_llm_with_retry():
    return ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.4,
        max_tokens=200,
        google_api_key=GEMINI_API_KEY
    )

llm = create_llm_with_retry()

# Prompt setup
system_prompt = (
    "You are a highly experienced and qualified mental health counsellor for high school students. "
    "Use the retrieved context and the user’s prompt to answer, give advice, and suggest things to ease the problem of the user. "
    "Use an empathetic and warm tone to not excite any reaction but to support the user. "
    "Take a deep breath and process all the context, then give an answer which best just advises and suggests rather than straight up solving the problem. "
    "Ask questions and ask for more context if required. "
    "You are the single thing supporting and comforting the user in any given situation, so tread lightly so as to not get too personal yet act as a warm counsellor to take care of the mental health of the user. "
    "If you don’t have straight answers or if the answers are too complex, start by recommending to seek warmth and comfort from adults, teachers, counsellors, or booking an appointment with the student therapist. "
    "Give answers in simple wording, short and friendly manner. "
    "Use a suggestive style and avoid making it sound compulsory or like an order. "
    "Take time to give answers and don’t make up information entirely. Be kind and warm. "
    "Ensure all replies have complete sentences and a full-stop is present at the end of the last sentence. "
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=10),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(google.api_core.exceptions.ResourceExhausted)
)
def invoke_rag_chain(input_text):
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)
    return rag_chain.invoke({"input": input_text})

# response = invoke_rag_chain("I am so down right now. I dont want to live")
response = invoke_rag_chain("I dont feel good today because i dont have a girfriend")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")


I understand you're feeling down because you don't have a girlfriend right now. It's completely normal to feel lonely or like you're missing out when you desire a romantic connection.

Have you felt this way for a while, or is it more recent? Sometimes, exploring the root of these feelings can help. Are there specific things that trigger this feeling, like seeing other couples or upcoming events?

In the meantime, remember that your worth isn't defined by your relationship status. There are so many other fulfilling aspects of life to focus on! Have you considered exploring new hobbies, spending time with friends and family, or focusing on personal goals? Sometimes, when we're busy and engaged in things we enjoy, those feelings of loneliness can lessen.

Also, remember the advice from the text, such as going out, trying new things, and appreciating what you have.


In [4]:
response = invoke_rag_chain("what if i love my gf but want to break up with her as im attracetd to other women")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")

That sounds like a really tough situation to be in, and it's completely understandable that you're feeling conflicted. It's not easy when your heart is being pulled in different directions.

Have you had a chance to really explore what it is about other women that's attracting you? Sometimes, understanding those feelings can give you more clarity about what you're truly seeking in a relationship.

Also, have you and your girlfriend talked about your feelings and expectations for the relationship? Sometimes, open and honest communication can help you both understand each other better and find a way forward, whether that's together or separately.

Remember, it's okay to take some time to figure out what's best for you. Talking to a trusted adult, like a school counselor, could also provide some helpful guidance as you navigate these complex emotions.


In [ ]:
response = invoke_rag_chain("Ayush touches me inappropriately and I feel sad and gropped, ptsd")

if "answer" in response:
    print(response["answer"])
else:
    print("No answer found.")